In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import re
import rushd as rd
import scipy as sp
import seaborn as sns

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.09.28_exp089.2'/'export'
base_path2 = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.10.17_exp089.3'/'export'
plates = pd.DataFrame({
    'data_path': [base_path/'293T', base_path/'iPS11', base_path2],
    'yaml_path': [base_path/'wells.yaml']*2 + [base_path2/'wells.yaml'],
    'cell': ['293T', 'iPS11', '293T'],
})
output_path = rd.rootdir/'output'/'KL_exp089.2_viral-titer-calculation'
cache_path = output_path/'exp089.2_viral-titer.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mGL-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

### Calculate titer
`rd.flow.moi` takes:

1. A DataFrame with the following columns:

    - condition
    - replicate
    - starting_cell_count
    - scaling (dilution factor relative to max_virus)
    - max_virus

2. Information to gate infected cells

    - color_column_name (channel to gate on)
    - color_column_cutoff (gate)

3. Optional parameters

    - output_path (where to save the plots)
    - summary_method (mean/median of replicates)

In [ ]:
# Create columns for rd.flow.moi
data.loc[data['cell']=='293T', 'starting_cell_count'] = 2e4
data.loc[data['cell']=='iPS11', 'starting_cell_count'] = 3e4

data['condition'] = data['construct']

In [ ]:
# Draw gates on uninfected population
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['condition']=='UI'].groupby(['biorep','cell'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

In [ ]:
plot_df = data[(data['condition']=='UI')]
x = 'mGL-A'
y = 'mRuby2-A'
g = sns.displot(data=plot_df, x=x, y=y, col='cell', kind='kde', row='biorep',
                log_scale=True, hue='construct', legend=False,
                common_norm=False, fill=False, levels=7, facet_kws=dict(margin_titles=True))

for (biorep,cell), ax in g.axes_dict.items():
    gate = gates[(gates['cell']==cell) & (gates['biorep']==biorep)]
    if gate.empty: continue
    ax.axvline(gate[x].values[0], color='black', zorder=0)
    ax.axhline(gate[y].values[0], color='black', zorder=0)

g.figure.savefig(rd.outfile(output_path/'kde_mGL-mRuby2_UI-gates.png'))

In [ ]:
for (biorep, cell), group in data.groupby(['biorep','cell']):

    plot_df = group[(group['scaling']==1)].groupby('condition').sample(1000)

    g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', col='condition', col_wrap=4, kind='kde',
                    log_scale=True, common_norm=False, fill=False, levels=7,
                    hue='construct')
    
    gate = gates[(gates['cell']==cell) & (gates['biorep']==biorep)]
    if gate.empty: continue

    for _, ax in g.axes_dict.items():
        ax.axvline(gate[x].values[0], color='black', zorder=0)
        ax.axhline(gate[y].values[0], color='black', zorder=0)

    g.figure.savefig(rd.outfile(output_path/f'kde_mGL-mRuby2_by-construct-with-gates_{cell}_biorep{biorep}.png'))

In [ ]:
# Categorize cells into quadrants based on two gates
# Possible values:
#   0 = double negative
#   1 = x-positive
#   2 = y-positive
#   3 = double positive
def get_quadrant(df, x, y, gates):
    gate_x = gates.loc[(gates['biorep']==df['biorep'].values[0]) & (gates['cell']==df['cell'].values[0]), x]
    gate_y = gates.loc[(gates['biorep']==df['biorep'].values[0]) & (gates['cell']==df['cell'].values[0]), y]
    df['x'] = data[x] > gate_x.values[0]
    df['y'] = data[y] > gate_y.values[0]
    df['quadrant'] = df['x'].astype(int) + df['y'].astype(int)*2
    return df

y = 'mGL-A'
x = 'mRuby2-A'
data = data.groupby(['biorep','cell'])[data.columns].apply(lambda df: get_quadrant(df, x, y, gates))
data.reset_index(drop=True, inplace=True)
display(data)

In [ ]:
# Calculate titer by gating on mGL+ cells
#   note that y = mGL above, so quadrants 2&3 are mGL+
df_titer = data.groupby(['cell','biorep'])[data.columns].apply(lambda df: rd.flow.moi(df, 'quadrant', 1.5))
df_titer.reset_index(inplace=True)
display(df_titer)

In [ ]:
# Save titers
df_titer_save = df_titer.drop(columns=['replicate', 'starting_cell_count','tui_ratio_per_vol','moi'])
df_titer_save.to_csv(output_path/'titer_mGL-mR2.csv')

In [ ]:
plot_df = data[(data['cell']=='293T')]
g = sns.displot(data=plot_df, x='mGL-A', hue='scaling', col='construct', row='biorep',
                common_norm=False, kind='kde', log_scale=True)

In [ ]:
biorep = 3
plot_df = data[(data['biorep']==biorep)].groupby('condition').sample(2000)

g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', col='condition', kind='kde', row='scaling',
                log_scale=True, common_norm=False, fill=False, levels=7, facet_kws=dict(margin_titles=True),
                hue='construct')

gate = gates[(gates['biorep']==biorep)]

for _, ax in g.axes_dict.items():
    ax.axvline(gate[x].values[0], color='black', zorder=0)
    ax.axhline(gate[y].values[0], color='black', zorder=0)